In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import random

class GoogleFormFiller:
    def __init__(self, form_url):
        self.driver = webdriver.Chrome()
        self.form_url = form_url
        self.wait = WebDriverWait(self.driver, 10)
    
    def open_form(self):
        """Mở Google Form"""
        self.driver.get(self.form_url)
        time.sleep(2)
    
    def fill_single_choice_questions(self):
        """Điền các câu hỏi single choice (radio button) với tỷ lệ cụ thể"""
        try:
            # Tìm tất cả radiogroup
            radiogroups = self.driver.find_elements(By.CSS_SELECTOR, '[role="radiogroup"]')
            
            for i, group in enumerate(radiogroups):
                try:
                    # Tìm tất cả radio buttons trong group này
                    radios = group.find_elements(By.CSS_SELECTOR, '[role="radio"]')
                    
                    # Lọc bỏ các option "Mục khác"
                    valid_radios = []
                    for radio in radios:
                        # Kiểm tra data-value để tránh __other_option__
                        data_value = radio.get_attribute('data-value')
                        if data_value != '__other_option__':
                            # Kiểm tra thêm aria-label và text content
                            label_text = radio.get_attribute('aria-label')
                            if label_text:
                                label_text = label_text.lower()
                                if 'khác' not in label_text and 'mục khác' not in label_text:
                                    valid_radios.append(radio)
                            else:
                                valid_radios.append(radio)
                    
                    selected = None
                    
                    # Xử lý đặc biệt cho từng câu hỏi
                    if i == 0:  # Câu hỏi đầu tiên (giới tính)
                        # Tìm option "Nữ"
                        female_option = None
                        for radio in valid_radios:
                            label = radio.get_attribute('aria-label') or radio.get_attribute('data-value') or ''
                            if 'nữ' in label.lower():
                                female_option = radio
                                break
                        
                        # Chọn "Nữ" với tỷ lệ 84%
                        if female_option and random.random() < 0.87:
                            selected = female_option
                            print("Chọn 'Nữ' (84% probability)")
                        else:
                            # Chọn ngẫu nhiên option khác (16%)
                            other_options = [r for r in valid_radios if r != female_option]
                            if other_options:
                                selected = random.choice(other_options)
                                print("Chọn option khác (13% probability)")
                            else:
                                selected = random.choice(valid_radios) if valid_radios else None
                    
                    elif i == 1:  # Câu hỏi độ tuổi
                        # Tìm các option độ tuổi
                        age_20_25 = None
                        age_26_35 = None
                        other_ages = []
                        
                        for radio in valid_radios:
                            label = radio.get_attribute('aria-label') or radio.get_attribute('data-value') or ''
                            if '20-25' in label:
                                age_20_25 = radio
                            elif '26-35' in label:
                                age_26_35 = radio
                            else:
                                other_ages.append(radio)
                        
                        # Phân phối tỷ lệ: 20-25 (35%), 26-35 (40%), còn lại (25%)
                        rand = random.random()
                        if age_20_25 and rand < 0.4:
                            selected = age_20_25
                            print("Chọn '20-25' (40% probability)")
                        elif age_26_35 and rand < 0.9:  # 35% + 40% = 75%
                            selected = age_26_35
                            print("Chọn '26-35' (50% probability)")
                        else:
                            # Chọn ngẫu nhiên từ các độ tuổi khác (10%)
                            if other_ages:
                                selected = random.choice(other_ages)
                                selected_label = selected.get_attribute('aria-label') or selected.get_attribute('data-value')
                                print(f"Chọn '{selected_label}' (25% probability)")
                            else:
                                selected = random.choice(valid_radios) if valid_radios else None
                    
                    # Kiểm tra nếu câu hỏi về con nhỏ dưới 3 tuổi
                    elif self.is_child_question(group):
                        # Tìm option "Có"
                        yes_option = None
                        for radio in valid_radios:
                            label = radio.get_attribute('aria-label') or radio.get_attribute('data-value') or ''
                            if label.lower().strip() == 'có':
                                yes_option = radio
                                break
                        
                        # Chọn "Có" với tỷ lệ 92%
                        if yes_option and random.random() < 0.92:
                            selected = yes_option
                            print("Chọn 'Có' cho câu hỏi con nhỏ (92% probability)")
                        else:
                            # Chọn "Không" (8%)
                            other_options = [r for r in valid_radios if r != yes_option]
                            if other_options:
                                selected = other_options[0]  # Thường là "Không"
                                print("Chọn 'Không' cho câu hỏi con nhỏ (8% probability)")
                            else:
                                selected = random.choice(valid_radios) if valid_radios else None
                    
                    # Kiểm tra nếu câu hỏi về Kamidi
                    elif self.is_kamidi_question(group):
                        # Tìm option "Chưa từng nghe"
                        never_heard_option = None
                        heard_option = None
                        used_option = None
                        
                        for radio in valid_radios:
                            label = radio.get_attribute('aria-label') or radio.get_attribute('data-value') or ''
                            label_lower = label.lower()
                            if 'chưa từng nghe' in label_lower or 'chưa nghe' in label_lower:
                                never_heard_option = radio
                            elif 'đã nghe' in label_lower:
                                heard_option = radio
                            elif 'đã sử dụng' in label_lower:
                                used_option = radio
                        
                        # Chọn "Chưa từng nghe" với tỷ lệ 13%
                        rand = random.random()
                        if never_heard_option and rand < 0.13:
                            selected = never_heard_option
                            print("Chọn 'Chưa từng nghe' cho Kamidi (13% probability)")
                        elif heard_option and rand < 0.75:  # 13% + 62% = 75% (giả sử "Đã nghe" chiếm 62%)
                            selected = heard_option
                            print("Chọn 'Đã nghe' cho Kamidi (62% probability)")
                        elif used_option:
                            selected = used_option
                            print("Chọn 'Đã sử dụng sản phẩm' cho Kamidi (25% probability)")
                        else:
                            selected = random.choice(valid_radios) if valid_radios else None
                    
                    # Kiểm tra nếu câu hỏi về quảng cáo Kamidi
                    elif self.is_kamidi_ad_question(group):
                        # Tìm option "Gây khó chịu"
                        annoying_option = None
                        other_options = []
                        
                        for radio in valid_radios:
                            label = radio.get_attribute('aria-label') or radio.get_attribute('data-value') or ''
                            if 'gây khó chịu' in label.lower():
                                annoying_option = radio
                            else:
                                other_options.append(radio)
                        
                        # Chọn "Gây khó chịu" với tỷ lệ 5%
                        if annoying_option and random.random() < 0.05:
                            selected = annoying_option
                            print("Chọn 'Gây khó chịu' cho quảng cáo Kamidi (5% probability)")
                        else:
                            # Chọn ngẫu nhiên option khác (95%)
                            if other_options:
                                selected = random.choice(other_options)
                                selected_label = selected.get_attribute('aria-label') or selected.get_attribute('data-value')
                                print(f"Chọn '{selected_label}' cho quảng cáo Kamidi (95% probability)")
                            else:
                                selected = random.choice(valid_radios) if valid_radios else None
                    
                    else:
                        # Câu hỏi khác: chọn ngẫu nhiên bình thường
                        selected = random.choice(valid_radios) if valid_radios else None
                    
                    # Click vào option đã chọn
                    if selected:
                        self.driver.execute_script("arguments[0].click();", selected)
                        time.sleep(0.1)
                        selected_label = selected.get_attribute('aria-label') or selected.get_attribute('data-value')
                        print(f"Đã chọn: {selected_label}")
                        
                except Exception as e:
                    print(f"Lỗi khi điền single choice: {e}")
                    continue
                    
        except Exception as e:
            print(f"Không tìm thấy câu hỏi single choice: {e}")
    
    def is_child_question(self, group):
        """Kiểm tra xem có phải câu hỏi về con nhỏ dưới 3 tuổi không"""
        try:
            # Tìm element chứa tiêu đề câu hỏi trong cùng container
            parent = group
            for _ in range(5):  # Tìm kiếm trong 5 level parent
                question_elements = parent.find_elements(By.CSS_SELECTOR, '.M7eMe, [role="heading"]')
                for elem in question_elements:
                    text = elem.text.lower()
                    if 'con nhỏ' in text and '3 tuổi' in text:
                        return True
                parent = parent.find_element(By.XPATH, '..')
            return False
        except:
            return False
    
    def is_kamidi_question(self, group):
        """Kiểm tra xem có phải câu hỏi về thương hiệu Kamidi không"""
        try:
            # Tìm element chứa tiêu đề câu hỏi trong cùng container
            parent = group
            for _ in range(5):  # Tìm kiếm trong 5 level parent
                question_elements = parent.find_elements(By.CSS_SELECTOR, '.M7eMe, [role="heading"]')
                for elem in question_elements:
                    text = elem.text.lower()
                    if 'kamidi' in text and ('nghe' in text or 'thương hiệu' in text):
                        return True
                parent = parent.find_element(By.XPATH, '..')
            return False
        except:
            return False
    
    def is_kamidi_ad_question(self, group):
        """Kiểm tra xem có phải câu hỏi về quảng cáo Kamidi không"""
        try:
            # Tìm element chứa tiêu đề câu hỏi trong cùng container
            parent = group
            for _ in range(5):  # Tìm kiếm trong 5 level parent
                question_elements = parent.find_elements(By.CSS_SELECTOR, '.M7eMe, [role="heading"]')
                for elem in question_elements:
                    text = elem.text.lower()
                    if 'kamidi' in text and 'quảng cáo' in text:
                        return True
                parent = parent.find_element(By.XPATH, '..')
            return False
        except:
            return False
    
    def fill_multiple_choice_questions(self):
        """Điền các câu hỏi multiple choice (checkbox) với tỷ lệ đặc biệt cho kênh mua hàng"""
        try:
            # Tìm tất cả checkbox questions
            checkbox_lists = self.driver.find_elements(By.CSS_SELECTOR, '[role="list"]')
            
            for list_elem in checkbox_lists:
                try:
                    # Kiểm tra xem có phải là checkbox list không
                    checkboxes = list_elem.find_elements(By.CSS_SELECTOR, '[role="checkbox"]')
                    
                    if not checkboxes:
                        continue
                    
                    # Lọc bỏ các option "Mục khác"
                    valid_checkboxes = []
                    for checkbox in checkboxes:
                        # Kiểm tra data-answer-value để tránh __other_option__
                        data_value = checkbox.get_attribute('data-answer-value')
                        if data_value != '__other_option__':
                            # Kiểm tra thêm aria-label
                            label_text = checkbox.get_attribute('aria-label')
                            if label_text:
                                label_text = label_text.lower()
                                if 'khác' not in label_text and 'mục khác' not in label_text:
                                    valid_checkboxes.append(checkbox)
                            else:
                                valid_checkboxes.append(checkbox)
                    
                    # Kiểm tra nếu đây là câu hỏi về kênh mua sản phẩm
                    if self.is_shopping_channel_question(list_elem):
                        self.fill_shopping_channel_checkboxes(valid_checkboxes)
                    else:
                        # Chọn ngẫu nhiên 1-3 options hợp lệ cho câu hỏi khác
                        if valid_checkboxes:
                            num_to_select = random.randint(1, min(3, len(valid_checkboxes)))
                            selected_checkboxes = random.sample(valid_checkboxes, num_to_select)
                            
                            for checkbox in selected_checkboxes:
                                self.driver.execute_script("arguments[0].click();", checkbox)
                                time.sleep(0.3)
                                print(f"Đã chọn: {checkbox.get_attribute('aria-label') or checkbox.get_attribute('data-answer-value')}")
                            
                except Exception as e:
                    print(f"Lỗi khi điền multiple choice: {e}")
                    continue
                    
        except Exception as e:
            print(f"Không tìm thấy câu hỏi multiple choice: {e}")
    
    def is_shopping_channel_question(self, list_elem):
        """Kiểm tra xem có phải câu hỏi về kênh mua sản phẩm không"""
        try:
            # Tìm element chứa tiêu đề câu hỏi trong cùng container
            parent = list_elem
            for _ in range(5):  # Tìm kiếm trong 5 level parent
                question_elements = parent.find_elements(By.CSS_SELECTOR, '.M7eMe, [role="heading"]')
                for elem in question_elements:
                    text = elem.text.lower()
                    if ('kênh' in text and 'mua' in text) or ('mua sản phẩm' in text and 'đâu' in text):
                        return True
                parent = parent.find_element(By.XPATH, '..')
            return False
        except:
            return False
    
    def fill_shopping_channel_checkboxes(self, checkboxes):
        """Điền câu hỏi kênh mua sản phẩm với tỷ lệ cụ thể"""
        try:
            # Tìm các option cụ thể
            shopee_option = None
            facebook_option = None
            store_option = None
            website_option = None
            other_options = []
            
            for checkbox in checkboxes:
                label = checkbox.get_attribute('aria-label') or checkbox.get_attribute('data-answer-value') or ''
                label_lower = label.lower()
                
                if 'shopee' in label_lower or 'lazada' in label_lower or 'tiktok' in label_lower:
                    shopee_option = checkbox
                elif 'facebook' in label_lower:
                    facebook_option = checkbox
                elif 'cửa hàng' in label_lower:
                    store_option = checkbox
                elif 'website' in label_lower:
                    website_option = checkbox
                else:
                    other_options.append(checkbox)
            
            # Áp dụng tỷ lệ: Shopee 63%, Facebook 17%, Cửa hàng 10%
            selected_options = []
            
            # Shopee/Lazada/Tiktok: 63%
            if shopee_option and random.random() < 0.63:
                selected_options.append(shopee_option)
                print("Chọn 'Shopee/Lazada/Tiktok' (63% probability)")
            
            # Facebook: 17%
            if facebook_option and random.random() < 0.17:
                selected_options.append(facebook_option)
                print("Chọn 'Facebook' (17% probability)")
            
            # Cửa hàng: 10%
            if store_option and random.random() < 0.15:
                selected_options.append(store_option)
                print("Chọn 'Cửa hàng' (15% probability)")
            
            # Website thương hiệu: 10% (còn lại)
            if website_option and random.random() < 0.10:
                selected_options.append(website_option)
                print("Chọn 'Website thương hiệu' (10% probability)")
            
            # Nếu không có option nào được chọn, chọn ít nhất 1 option
            if not selected_options:
                if shopee_option:  # Ưu tiên Shopee vì có tỷ lệ cao nhất
                    selected_options.append(shopee_option)
                    print("Chọn 'Shopee/Lazada/Tiktok' (fallback)")
                elif checkboxes:
                    selected_options.append(random.choice(checkboxes))
                    print("Chọn option ngẫu nhiên (fallback)")
            
            # Click vào các option đã chọn
            for checkbox in selected_options:
                self.driver.execute_script("arguments[0].click();", checkbox)
                time.sleep(0.3)
                
        except Exception as e:
            print(f"Lỗi khi điền câu hỏi kênh mua sản phẩm: {e}")
    
    def submit_form(self):
        """Submit form"""
        try:
            # Thử nhiều selector khác nhau cho nút submit
            submit_selectors = [
                '[aria-label="Submit"]',
                '[jsname="M2UYVd"]',
                'div[role="button"][aria-label="Submit"]',
                '.NPEfkd.RveJvd.snByac',
                'span:contains("Gửi")',
                '[type="submit"]'
            ]
            
            submit_button = None
            for selector in submit_selectors:
                try:
                    submit_button = self.wait.until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
                    )
                    break
                except TimeoutException:
                    continue
            
            if submit_button:
                submit_button.click()
                time.sleep(2)
                print("Form đã được submit thành công!")
                return True
            else:
                print("Không tìm thấy nút submit")
                return False
                
        except Exception as e:
            print(f"Lỗi khi submit form: {e}")
            return False
    
    def check_for_submit_another_response(self):
        """Kiểm tra và click vào nút 'Gửi phản hồi khác' nếu có"""
        try:
            # Tìm nút "Gửi phản hồi khác" hoặc "Submit another response"
            submit_another_selectors = [
                'a[href*="viewform"]:contains("Gửi phản hồi khác")',
                'a[href*="viewform"]',
                '.freebirdFormviewerViewResponseLinksContainer a',
                'a[aria-label*="another"]',
                'a[aria-label*="khác"]'
            ]
            
            for selector in submit_another_selectors:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for element in elements:
                        text = element.text.lower()
                        if 'gửi' in text and ('khác' in text or 'phản hồi' in text):
                            element.click()
                            time.sleep(2)
                            print("Đã click 'Gửi phản hồi khác'")
                            return True
                        elif 'submit another' in text:
                            element.click()
                            time.sleep(2)
                            print("Đã click 'Submit another response'")
                            return True
                except:
                    continue
                    
            # Nếu không tìm thấy, refresh lại trang form
            print("Không tìm thấy nút 'Gửi phản hồi khác', refresh lại form")
            self.driver.get(self.form_url)
            time.sleep(3)
            return True
            
        except Exception as e:
            print(f"Lỗi khi tìm nút 'Gửi phản hồi khác': {e}")
            # Fallback: refresh lại trang
            self.driver.get(self.form_url)
            time.sleep(3)
            return True
    
    def fill_form_once(self):
        """Điền form một lần"""
        try:
            # Điền single choice questions
            self.fill_single_choice_questions()
            
            # Điền multiple choice questions  
            self.fill_multiple_choice_questions()
            
            # Submit form
            success = self.submit_form()
            
            return success
            
        except Exception as e:
            print(f"Lỗi khi điền form: {e}")
            return False
    
    def fill_form_multiple_times(self, num_times, delay_between_forms=1):
        """
        Điền form nhiều lần
        
        Args:
            num_times (int): Số lần điền form
            delay_between_forms (int): Thời gian delay giữa các lần điền (giây)
        """
        print(f"Bắt đầu điền form {num_times} lần...")
        print("Tỷ lệ được áp dụng:")
        print("- Giới tính: Nữ 84%, Khác 16%")
        print("- Độ tuổi: 20-25 (35%), 26-35 (40%), Khác (25%)")
        print("- Con nhỏ dưới 3 tuổi: Có 92%, Không 8%")
        print("- Kênh mua sản phẩm: Shopee 63%, Facebook 17%, Cửa hàng 10%, Website 10%")
        print("- Đã nghe Kamidi: Chưa nghe 13%, Đã nghe 62%, Đã sử dụng 25%")
        print("- Quảng cáo Kamidi: Gây khó chịu 5%, Khác 95%")
        print("-" * 50)
        
        success_count = 0
        
        # Mở form lần đầu
        self.open_form()
        
        for i in range(num_times):
            print(f"\n--- Lần {i + 1}/{num_times} ---")
            
            try:
                # Điền form
                if self.fill_form_once():
                    success_count += 1
                    print(f"✓ Lần {i + 1} thành công")
                    
                    # Nếu chưa phải lần cuối cùng, chuẩn bị cho lần tiếp theo
                    if i < num_times - 1:
                        # Kiểm tra và click "Gửi phản hồi khác"
                        self.check_for_submit_another_response()
                        
                        # Delay trước khi điền lần tiếp theo
                        if delay_between_forms > 0:
                            print(f"Chờ {delay_between_forms} giây...")
                            time.sleep(delay_between_forms)
                else:
                    print(f"✗ Lần {i + 1} thất bại")
                    
                    # Thử refresh lại form
                    self.driver.get(self.form_url)
                    time.sleep(3)
                    
            except Exception as e:
                print(f"✗ Lỗi ở lần {i + 1}: {e}")
                # Thử refresh lại form
                try:
                    self.driver.get(self.form_url)
                    time.sleep(3)
                except:
                    pass
        
        print(f"\n=== KẾT QUẢ ===")
        print(f"Tổng số lần thành công: {success_count}/{num_times}")
        print(f"Tỷ lệ thành công: {success_count/num_times*100:.1f}%")
    
    def fill_form(self):
        """Điền toàn bộ form (giữ lại để tương thích ngược)"""
        print("Bắt đầu điền form...")
        
        # Mở form
        self.open_form()
        
        # Điền form một lần
        self.fill_form_once()
    
    def close(self):
        """Đóng browser"""
        self.driver.quit()

# Hàm tiện ích để điền form nhiều lần
def fill_google_form_multiple_times(form_url, num_times, delay_between_forms=3):
    """
    Hàm tiện ích để điền Google Form nhiều lần
    
    Args:
        form_url (str): URL của Google Form
        num_times (int): Số lần điền form
        delay_between_forms (int): Thời gian delay giữa các lần điền (giây)
    """
    filler = GoogleFormFiller(form_url)
    
    try:
        filler.fill_form_multiple_times(num_times, delay_between_forms)
        input("\nNhấn Enter để đóng browser...")
        
    except KeyboardInterrupt:
        print("\nDừng chương trình theo yêu cầu người dùng")
        
    except Exception as e:
        print(f"Lỗi: {e}")
        
    finally:
        filler.close()

# Sử dụng
if __name__ == "__main__":
    # Thay URL form của bạn vào đây
    FORM_URL = "https://docs.google.com/forms/d/e/1FAIpQLScL2puWWWKJ5s8D3eeD3KZ7pbDsAY8X4N-40WrZso1a2_SwbQ/viewform"
    
    # Điền 200 lần với delay 2 giây
    fill_google_form_multiple_times(FORM_URL, 200, 2)

Bắt đầu điền form 200 lần...
Tỷ lệ được áp dụng:
- Giới tính: Nữ 84%, Khác 16%
- Độ tuổi: 20-25 (35%), 26-35 (40%), Khác (25%)
- Con nhỏ dưới 3 tuổi: Có 92%, Không 8%
- Kênh mua sản phẩm: Shopee 63%, Facebook 17%, Cửa hàng 10%, Website 10%
- Đã nghe Kamidi: Chưa nghe 13%, Đã nghe 62%, Đã sử dụng 25%
- Quảng cáo Kamidi: Gây khó chịu 5%, Khác 95%
--------------------------------------------------

--- Lần 1/200 ---
Chọn 'Nữ' (84% probability)
Đã chọn: Nữ
Chọn '26-35' (40% probability)
Đã chọn: 26-35
Chọn 'Có' cho câu hỏi con nhỏ (92% probability)
Đã chọn: Có
Chọn 'Đã nghe' cho Kamidi (62% probability)
Đã chọn: Đã nghe
Chọn 'Không để ý' cho quảng cáo Kamidi (95% probability)
Đã chọn: Không để ý
Đã chọn: Thỉnh thoảng
Đã chọn: Không
Đã chọn: Đồng ý
Đã chọn: Rồi
Đã chọn: Không hài lòng
Đã chọn: Có
Đã chọn: Thiếu chuyên môn
Đã chọn: Google
Đã chọn: Không
Đã chọn: 4- Nắm khá rõ thương hiệu, biết nhiều dòng sản phẩm
Chọn 'Shopee/Lazada/Tiktok' (63% probability)
Chọn 'Facebook' (17% probability)

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import random

class GoogleFormFiller:
    def __init__(self, form_url):
        self.driver = webdriver.Chrome()
        self.form_url = form_url
        self.wait = WebDriverWait(self.driver, 10)
    
    def open_form(self):
        """Mở Google Form"""
        self.driver.get(self.form_url)
        time.sleep(2)
    
    def fill_single_choice_questions(self):
        """Điền các câu hỏi single choice (radio button) với tỷ lệ cụ thể"""
        try:
            # Tìm tất cả radiogroup
            radiogroups = self.driver.find_elements(By.CSS_SELECTOR, '[role="radiogroup"]')
            
            for i, group in enumerate(radiogroups):
                try:
                    # Tìm tất cả radio buttons trong group này
                    radios = group.find_elements(By.CSS_SELECTOR, '[role="radio"]')
                    
                    # Lọc bỏ các option "Mục khác"
                    valid_radios = []
                    for radio in radios:
                        # Kiểm tra data-value để tránh __other_option__
                        data_value = radio.get_attribute('data-value')
                        if data_value != '__other_option__':
                            # Kiểm tra thêm aria-label và text content
                            label_text = radio.get_attribute('aria-label')
                            if label_text:
                                label_text = label_text.lower()
                                if 'khác' not in label_text and 'mục khác' not in label_text:
                                    valid_radios.append(radio)
                            else:
                                valid_radios.append(radio)
                    
                    selected = None
                    
                    # Xử lý đặc biệt cho từng câu hỏi
                    if i == 0:  # Câu hỏi đầu tiên (giới tính)
                        # Tìm option "Nữ"
                        female_option = None
                        for radio in valid_radios:
                            label = radio.get_attribute('aria-label') or radio.get_attribute('data-value') or ''
                            if 'nữ' in label.lower():
                                female_option = radio
                                break
                        
                        # Chọn "Nữ" với tỷ lệ 84%
                        if female_option and random.random() < 0.84:
                            selected = female_option
                            print("Chọn 'Nữ' (84% probability)")
                        else:
                            # Chọn ngẫu nhiên option khác (16%)
                            other_options = [r for r in valid_radios if r != female_option]
                            if other_options:
                                selected = random.choice(other_options)
                                print("Chọn option khác (16% probability)")
                            else:
                                selected = random.choice(valid_radios) if valid_radios else None
                    
                    elif i == 1:  # Câu hỏi độ tuổi
                        # Tìm các option độ tuổi
                        age_20_25 = None
                        age_26_35 = None
                        other_ages = []
                        
                        for radio in valid_radios:
                            label = radio.get_attribute('aria-label') or radio.get_attribute('data-value') or ''
                            if '20-25' in label:
                                age_20_25 = radio
                            elif '26-35' in label:
                                age_26_35 = radio
                            else:
                                other_ages.append(radio)
                        
                        # Phân phối tỷ lệ: 20-25 (35%), 26-35 (40%), còn lại (25%)
                        rand = random.random()
                        if age_20_25 and rand < 0.35:
                            selected = age_20_25
                            print("Chọn '20-25' (35% probability)")
                        elif age_26_35 and rand < 0.75:  # 35% + 40% = 75%
                            selected = age_26_35
                            print("Chọn '26-35' (40% probability)")
                        else:
                            # Chọn ngẫu nhiên từ các độ tuổi khác (25%)
                            if other_ages:
                                selected = random.choice(other_ages)
                                selected_label = selected.get_attribute('aria-label') or selected.get_attribute('data-value')
                                print(f"Chọn '{selected_label}' (25% probability)")
                            else:
                                selected = random.choice(valid_radios) if valid_radios else None
                    
                    # Kiểm tra nếu câu hỏi về con nhỏ dưới 3 tuổi
                    elif self.is_child_question(group):
                        # Tìm option "Có"
                        yes_option = None
                        for radio in valid_radios:
                            label = radio.get_attribute('aria-label') or radio.get_attribute('data-value') or ''
                            if label.lower().strip() == 'có':
                                yes_option = radio
                                break
                        
                        # Chọn "Có" với tỷ lệ 92%
                        if yes_option and random.random() < 0.92:
                            selected = yes_option
                            print("Chọn 'Có' cho câu hỏi con nhỏ (92% probability)")
                        else:
                            # Chọn "Không" (8%)
                            other_options = [r for r in valid_radios if r != yes_option]
                            if other_options:
                                selected = other_options[0]  # Thường là "Không"
                                print("Chọn 'Không' cho câu hỏi con nhỏ (8% probability)")
                            else:
                                selected = random.choice(valid_radios) if valid_radios else None
                    
                    # Kiểm tra nếu câu hỏi về Kamidi
                    elif self.is_kamidi_question(group):
                        # Tìm option "Chưa từng nghe"
                        never_heard_option = None
                        heard_option = None
                        used_option = None
                        
                        for radio in valid_radios:
                            label = radio.get_attribute('aria-label') or radio.get_attribute('data-value') or ''
                            label_lower = label.lower()
                            if 'chưa từng nghe' in label_lower or 'chưa nghe' in label_lower:
                                never_heard_option = radio
                            elif 'đã nghe' in label_lower:
                                heard_option = radio
                            elif 'đã sử dụng' in label_lower:
                                used_option = radio
                        
                        # Chọn "Chưa từng nghe" với tỷ lệ 13%
                        rand = random.random()
                        if never_heard_option and rand < 0.13:
                            selected = never_heard_option
                            print("Chọn 'Chưa từng nghe' cho Kamidi (13% probability)")
                        elif heard_option and rand < 0.75:  # 13% + 62% = 75% (giả sử "Đã nghe" chiếm 62%)
                            selected = heard_option
                            print("Chọn 'Đã nghe' cho Kamidi (62% probability)")
                        elif used_option:
                            selected = used_option
                            print("Chọn 'Đã sử dụng sản phẩm' cho Kamidi (25% probability)")
                        else:
                            selected = random.choice(valid_radios) if valid_radios else None
                    
                    # Kiểm tra nếu câu hỏi về quảng cáo Kamidi thu hút
                    elif self.is_kamidi_attraction_question(group):
                        # Tìm các option
                        very_attractive = None
                        normal = None
                        no_attention = None
                        annoying = None
                        
                        for radio in valid_radios:
                            label = radio.get_attribute('aria-label') or radio.get_attribute('data-value') or ''
                            label_lower = label.lower()
                            if 'rất thu hút' in label_lower:
                                very_attractive = radio
                            elif 'bình thường' in label_lower:
                                normal = radio
                            elif 'không để ý' in label_lower:
                                no_attention = radio
                            elif 'gây khó chịu' in label_lower:
                                annoying = radio
                        
                        # Phân phối tỷ lệ: Gây khó chịu 2%, Không để ý 30%, còn lại phân bố cho Rất thu hút và Bình thường
                        rand = random.random()
                        if annoying and rand < 0.02:
                            selected = annoying
                            print("Chọn 'Gây khó chịu' cho quảng cáo Kamidi (2% probability)")
                        elif no_attention and rand < 0.32:  # 2% + 30% = 32%
                            selected = no_attention
                            print("Chọn 'Không để ý' cho quảng cáo Kamidi (30% probability)")
                        elif very_attractive and rand < 0.66:  # Giả sử 'Rất thu hút' chiếm 34%
                            selected = very_attractive
                            print("Chọn 'Rất thu hút' cho quảng cáo Kamidi (34% probability)")
                        elif normal:
                            selected = normal
                            print("Chọn 'Bình thường' cho quảng cáo Kamidi (34% probability)")
                        else:
                            selected = random.choice(valid_radios) if valid_radios else None
                    
                    # Kiểm tra nếu câu hỏi về xem livestream/video review Kamidi
                    elif self.is_kamidi_content_question(group):
                        # Tìm option "Có"
                        yes_option = None
                        no_option = None
                        
                        for radio in valid_radios:
                            label = radio.get_attribute('aria-label') or radio.get_attribute('data-value') or ''
                            label_lower = label.lower()
                            if label_lower.strip() == 'có':
                                yes_option = radio
                            elif label_lower.strip() == 'không':
                                no_option = radio
                        
                        # Chọn "Có" với tỷ lệ 87%
                        if yes_option and random.random() < 0.87:
                            selected = yes_option
                            print("Chọn 'Có' cho xem livestream/video Kamidi (87% probability)")
                        elif no_option:
                            selected = no_option
                            print("Chọn 'Không' cho xem livestream/video Kamidi (13% probability)")
                        else:
                            selected = random.choice(valid_radios) if valid_radios else None
                    
                    else:
                        # Câu hỏi khác: chọn ngẫu nhiên bình thường
                        selected = random.choice(valid_radios) if valid_radios else None
                    
                    # Click vào option đã chọn
                    if selected:
                        self.driver.execute_script("arguments[0].click();", selected)
                        time.sleep(0.5)
                        selected_label = selected.get_attribute('aria-label') or selected.get_attribute('data-value')
                        print(f"Đã chọn: {selected_label}")
                        
                except Exception as e:
                    print(f"Lỗi khi điền single choice: {e}")
                    continue
                    
        except Exception as e:
            print(f"Không tìm thấy câu hỏi single choice: {e}")
    
    def is_child_question(self, group):
        """Kiểm tra xem có phải câu hỏi về con nhỏ dưới 3 tuổi không"""
        try:
            # Tìm element chứa tiêu đề câu hỏi trong cùng container
            parent = group
            for _ in range(5):  # Tìm kiếm trong 5 level parent
                question_elements = parent.find_elements(By.CSS_SELECTOR, '.M7eMe, [role="heading"]')
                for elem in question_elements:
                    text = elem.text.lower()
                    if 'con nhỏ' in text and '3 tuổi' in text:
                        return True
                parent = parent.find_element(By.XPATH, '..')
            return False
        except:
            return False
    
    def is_kamidi_question(self, group):
        """Kiểm tra xem có phải câu hỏi về thương hiệu Kamidi không"""
        try:
            # Tìm element chứa tiêu đề câu hỏi trong cùng container
            parent = group
            for _ in range(5):  # Tìm kiếm trong 5 level parent
                question_elements = parent.find_elements(By.CSS_SELECTOR, '.M7eMe, [role="heading"]')
                for elem in question_elements:
                    text = elem.text.lower()
                    if 'kamidi' in text and ('nghe' in text or 'thương hiệu' in text) and 'thu hút' not in text:
                        return True
                parent = parent.find_element(By.XPATH, '..')
            return False
        except:
            return False
    
    def is_kamidi_attraction_question(self, group):
        """Kiểm tra xem có phải câu hỏi về quảng cáo Kamidi có thu hút không"""
        try:
            # Tìm element chứa tiêu đề câu hỏi trong cùng container
            parent = group
            for _ in range(5):  # Tìm kiếm trong 5 level parent
                question_elements = parent.find_elements(By.CSS_SELECTOR, '.M7eMe, [role="heading"]')
                for elem in question_elements:
                    text = elem.text.lower()
                    if 'kamidi' in text and ('thu hút' in text or 'quảng cáo' in text):
                        return True
                parent = parent.find_element(By.XPATH, '..')
            return False
        except:
            return False
    
    def is_kamidi_content_question(self, group):
        """Kiểm tra xem có phải câu hỏi về xem livestream/video review Kamidi không"""
        try:
            # Tìm element chứa tiêu đề câu hỏi trong cùng container
            parent = group
            for _ in range(5):  # Tìm kiếm trong 5 level parent
                question_elements = parent.find_elements(By.CSS_SELECTOR, '.M7eMe, [role="heading"]')
                for elem in question_elements:
                    text = elem.text.lower()
                    if 'kamidi' in text and ('livestream' in text or 'video' in text or 'review' in text):
                        return True
                parent = parent.find_element(By.XPATH, '..')
            return False
        except:
            return False
    
    def fill_multiple_choice_questions(self):
        """Điền các câu hỏi multiple choice (checkbox) với tỷ lệ đặc biệt cho kênh mua hàng và cải thiện truyền thông"""
        try:
            # Tìm tất cả checkbox questions
            checkbox_lists = self.driver.find_elements(By.CSS_SELECTOR, '[role="list"]')
            
            for list_elem in checkbox_lists:
                try:
                    # Kiểm tra xem có phải là checkbox list không
                    checkboxes = list_elem.find_elements(By.CSS_SELECTOR, '[role="checkbox"]')
                    
                    if not checkboxes:
                        continue
                    
                    # Lọc bỏ các option "Mục khác"
                    valid_checkboxes = []
                    for checkbox in checkboxes:
                        # Kiểm tra data-answer-value để tránh __other_option__
                        data_value = checkbox.get_attribute('data-answer-value')
                        if data_value != '__other_option__':
                            # Kiểm tra thêm aria-label
                            label_text = checkbox.get_attribute('aria-label')
                            if label_text:
                                label_text = label_text.lower()
                                if 'khác' not in label_text and 'mục khác' not in label_text:
                                    valid_checkboxes.append(checkbox)
                            else:
                                valid_checkboxes.append(checkbox)
                    
                    # Kiểm tra nếu đây là câu hỏi về kênh mua sản phẩm
                    if self.is_shopping_channel_question(list_elem):
                        self.fill_shopping_channel_checkboxes(valid_checkboxes)
                    # Kiểm tra nếu đây là câu hỏi về cải thiện truyền thông Kamidi
                    elif self.is_kamidi_improvement_question(list_elem):
                        self.fill_kamidi_improvement_checkboxes(valid_checkboxes)
                    else:
                        # Chọn ngẫu nhiên 1-3 options hợp lệ cho câu hỏi khác
                        if valid_checkboxes:
                            num_to_select = random.randint(1, min(3, len(valid_checkboxes)))
                            selected_checkboxes = random.sample(valid_checkboxes, num_to_select)
                            
                            for checkbox in selected_checkboxes:
                                self.driver.execute_script("arguments[0].click();", checkbox)
                                time.sleep(0.3)
                                print(f"Đã chọn: {checkbox.get_attribute('aria-label') or checkbox.get_attribute('data-answer-value')}")
                            
                except Exception as e:
                    print(f"Lỗi khi điền multiple choice: {e}")
                    continue
                    
        except Exception as e:
            print(f"Không tìm thấy câu hỏi multiple choice: {e}")
    
    def is_shopping_channel_question(self, list_elem):
        """Kiểm tra xem có phải câu hỏi về kênh mua sản phẩm không"""
        try:
            # Tìm element chứa tiêu đề câu hỏi trong cùng container
            parent = list_elem
            for _ in range(5):  # Tìm kiếm trong 5 level parent
                question_elements = parent.find_elements(By.CSS_SELECTOR, '.M7eMe, [role="heading"]')
                for elem in question_elements:
                    text = elem.text.lower()
                    if ('kênh' in text and 'mua' in text) or ('mua sản phẩm' in text and 'đâu' in text):
                        return True
                parent = parent.find_element(By.XPATH, '..')
            return False
        except:
            return False
    
    def is_kamidi_improvement_question(self, list_elem):
        """Kiểm tra xem có phải câu hỏi về cải thiện truyền thông Kamidi không"""
        try:
            # Tìm element chứa tiêu đề câu hỏi trong cùng container
            parent = list_elem
            for _ in range(5):  # Tìm kiếm trong 5 level parent
                question_elements = parent.find_elements(By.CSS_SELECTOR, '.M7eMe, [role="heading"]')
                for elem in question_elements:
                    text = elem.text.lower()
                    if 'kamidi' in text and ('cải thiện' in text or 'truyền thông' in text):
                        return True
                parent = parent.find_element(By.XPATH, '..')
            return False
        except:
            return False
    
    def fill_shopping_channel_checkboxes(self, checkboxes):
        """Điền câu hỏi kênh mua sản phẩm với tỷ lệ cụ thể"""
        try:
            # Tìm các option cụ thể
            shopee_option = None
            facebook_option = None
            store_option = None
            website_option = None
            other_options = []
            
            for checkbox in checkboxes:
                label = checkbox.get_attribute('aria-label') or checkbox.get_attribute('data-answer-value') or ''
                label_lower = label.lower()
                
                if 'shopee' in label_lower or 'lazada' in label_lower or 'tiktok' in label_lower:
                    shopee_option = checkbox
                elif 'facebook' in label_lower:
                    facebook_option = checkbox
                elif 'cửa hàng' in label_lower:
                    store_option = checkbox
                elif 'website' in label_lower:
                    website_option = checkbox
                else:
                    other_options.append(checkbox)
            
            # Áp dụng tỷ lệ: Shopee 63%, Facebook 17%, Cửa hàng 10%
            selected_options = []
            
            # Shopee/Lazada/Tiktok: 63%
            if shopee_option and random.random() < 0.63:
                selected_options.append(shopee_option)
                print("Chọn 'Shopee/Lazada/Tiktok' (63% probability)")
            
            # Facebook: 17%
            if facebook_option and random.random() < 0.17:
                selected_options.append(facebook_option)
                print("Chọn 'Facebook' (17% probability)")
            
            # Cửa hàng: 10%
            if store_option and random.random() < 0.10:
                selected_options.append(store_option)
                print("Chọn 'Cửa hàng' (10% probability)")
            
            # Website thương hiệu: 10% (còn lại)
            if website_option and random.random() < 0.10:
                selected_options.append(website_option)
                print("Chọn 'Website thương hiệu' (10% probability)")
            
            # Nếu không có option nào được chọn, chọn ít nhất 1 option
            if not selected_options:
                if shopee_option:  # Ưu tiên Shopee vì có tỷ lệ cao nhất
                    selected_options.append(shopee_option)
                    print("Chọn 'Shopee/Lazada/Tiktok' (fallback)")
                elif checkboxes:
                    selected_options.append(random.choice(checkboxes))
                    print("Chọn option ngẫu nhiên (fallback)")
            
            # Click vào các option đã chọn
            for checkbox in selected_options:
                self.driver.execute_script("arguments[0].click();", checkbox)
                time.sleep(0.3)
                
        except Exception as e:
            print(f"Lỗi khi điền câu hỏi kênh mua sản phẩm: {e}")
    
    def fill_kamidi_improvement_checkboxes(self, checkboxes):
        """Điền câu hỏi cải thiện truyền thông Kamidi với tỷ lệ cụ thể"""
        try:
            # Tìm các option cụ thể
            social_media_option = None
            content_option = None
            celebrity_option = None
            minigame_option = None
            feedback_option = None
            image_option = None
            website_option = None
            message_option = None
            ecommerce_option = None
            
            for checkbox in checkboxes:
                label = checkbox.get_attribute('aria-label') or checkbox.get_attribute('data-answer-value') or ''
                label_lower = label.lower()
                
                if 'tăng tần suất' in label_lower and 'mạng xã hội' in label_lower:
                    social_media_option = checkbox
                elif 'đầu tư nội dung' in label_lower:
                    content_option = checkbox
                elif 'người nổi tiếng' in label_lower or 'kol' in label_lower:
                    celebrity_option = checkbox
                elif 'mini game' in label_lower or 'khuyến mãi' in label_lower:
                    minigame_option = checkbox
                elif 'phản hồi thực tế' in label_lower:
                    feedback_option = checkbox
                elif 'hình ảnh sản phẩm' in label_lower:
                    image_option = checkbox
                elif 'website' in label_lower and 'dễ dùng' in label_lower:
                    website_option = checkbox
                elif 'thông điệp gần gũi' in label_lower:
                    message_option = checkbox
                elif 'sàn tmđt' in label_lower or ('shopee' in label_lower and 'lazada' in label_lower):
                    ecommerce_option = checkbox
            
            # Áp dụng tỷ lệ theo yêu cầu
            selected_options = []
            
            # Tăng tần suất xuất hiện trên mạng xã hội: 33%
            if social_media_option and random.random() < 0.33:
                selected_options.append(social_media_option)
                print("Chọn 'Tăng tần suất xuất hiện trên mạng xã hội' (33% probability)")
            
            # Tổ chức mini game / chương trình khuyến mãi: 20%
            if minigame_option and random.random() < 0.20:
                selected_options.append(minigame_option)
                print("Chọn 'Tổ chức mini game / chương trình khuyến mãi' (20% probability)")
            
            # Chia sẻ nhiều hơn về phản hồi thực tế: 15%
            if feedback_option and random.random() < 0.15:
                selected_options.append(feedback_option)
                print("Chọn 'Chia sẻ nhiều hơn về phản hồi thực tế' (15% probability)")
            
            # Liên kết với các sàn TMĐT: 23%
            if ecommerce_option and random.random() < 0.23:
                selected_options.append(ecommerce_option)
                print("Chọn 'Liên kết với các sàn TMĐT' (23% probability)")
            
            # Các option còn lại được phân bổ đều trong 9% còn lại (khoảng 2.25% mỗi option)
            remaining_options = [content_option, celebrity_option, image_option, website_option, message_option]
            for option in remaining_options:
                if option and random.random() < 0.0225:  # 2.25%
                    selected_options.append(option)
                    option_label = option.get_attribute('aria-label') or option.get_attribute('data-answer-value')
                    print(f"Chọn '{option_label}' (2.25% probability)")
            
            # Nếu không có option nào được chọn, chọn ít nhất 1 option theo tỷ lệ cao nhất
            if not selected_options:
                if social_media_option:  # Ưu tiên option có tỷ lệ cao nhất
                    selected_options.append(social_media_option)
                    print("Chọn 'Tăng tần suất xuất hiện trên mạng xã hội' (fallback)")
                elif checkboxes:
                    selected_options.append(random.choice(checkboxes))
                    print("Chọn option ngẫu nhiên (fallback)")
            
            # Click vào các option đã chọn
            for checkbox in selected_options:
                self.driver.execute_script("arguments[0].click();", checkbox)
                time.sleep(0.3)
                
        except Exception as e:
            print(f"Lỗi khi điền câu hỏi cải thiện truyền thông Kamidi: {e}")
    
    def submit_form(self):
        """Submit form"""
        try:
            # Thử nhiều selector khác nhau cho nút submit
            submit_selectors = [
                '[aria-label="Submit"]',
                '[jsname="M2UYVd"]',
                'div[role="button"][aria-label="Submit"]',
                '.NPEfkd.RveJvd.snByac',
                'span:contains("Gửi")',
                '[type="submit"]'
            ]
            
            submit_button = None
            for selector in submit_selectors:
                try:
                    submit_button = self.wait.until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
                    )
                    break
                except TimeoutException:
                    continue
            
            if submit_button:
                submit_button.click()
                time.sleep(2)
                print("Form đã được submit thành công!")
                return True
            else:
                print("Không tìm thấy nút submit")
                return False
                
        except Exception as e:
            print(f"Lỗi khi submit form: {e}")
            return False
    
    def check_for_submit_another_response(self):
        """Kiểm tra và click vào nút 'Gửi phản hồi khác' nếu có"""
        try:
            # Tìm nút "Gửi phản hồi khác" hoặc "Submit another response"
            submit_another_selectors = [
                'a[href*="viewform"]:contains("Gửi phản hồi khác")',
                'a[href*="viewform"]',
                '.freebirdFormviewerViewResponseLinksContainer a',
                'a[aria-label*="another"]',
                'a[aria-label*="khác"]'
            ]
            
            for selector in submit_another_selectors:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for element in elements:
                        text = element.text.lower()
                        if 'gửi' in text and ('khác' in text or 'phản hồi' in text):
                            element.click()
                            time.sleep(2)
                            print("Đã click 'Gửi phản hồi khác'")
                            return True
                        elif 'submit another' in text:
                            element.click()
                            time.sleep(2)
                            print("Đã click 'Submit another response'")
                            return True
                except:
                    continue
                    
            # Nếu không tìm thấy, refresh lại trang form
            print("Không tìm thấy nút 'Gửi phản hồi khác', refresh lại form")
            self.driver.get(self.form_url)
            time.sleep(3)
            return True
            
        except Exception as e:
            print(f"Lỗi khi tìm nút 'Gửi phản hồi khác': {e}")
            # Fallback: refresh lại trang
            self.driver.get(self.form_url)
            time.sleep(3)
            return True
    
    def fill_form_once(self):
        """Điền form một lần"""
        try:
            # Điền single choice questions
            self.fill_single_choice_questions()
            
            # Điền multiple choice questions  
            self.fill_multiple_choice_questions()
            
            # Submit form
            success = self.submit_form()
            
            return success
            
        except Exception as e:
            print(f"Lỗi khi điền form: {e}")
            return False
    
    def fill_form_multiple_times(self, num_times, delay_between_forms=1):
        """
        Điền form nhiều lần
        
        Args:
            num_times (int): Số lần điền form
            delay_between_forms (int): Thời gian delay giữa các lần điền (giây)
        """
        print(f"Bắt đầu điền form {num_times} lần...")
        print("Tỷ lệ được áp dụng:")
        print("- Giới tính: Nữ 84%, Khác 16%")
        print("- Độ tuổi: 20-25 (35%), 26-35 (40%), Khác (25%)")
        print("- Con nhỏ dưới 3 tuổi: Có 92%, Không 8%")
        print("- Kênh mua sản phẩm: Shopee 63%, Facebook 17%, Cửa hàng 10%, Website 10%")
        print("- Đã nghe Kamidi: Chưa nghe 13%, Đã nghe 62%, Đã sử dụng 25%")
        print("- Quảng cáo Kamidi: Gây khó chịu 5%, Khác 95%")
        print("-" * 50)
        
        success_count = 0
        
        # Mở form lần đầu
        self.open_form()
        
        for i in range(num_times):
            print(f"\n--- Lần {i + 1}/{num_times} ---")
            
            try:
                # Điền form
                if self.fill_form_once():
                    success_count += 1
                    print(f"✓ Lần {i + 1} thành công")
                    
                    # Nếu chưa phải lần cuối cùng, chuẩn bị cho lần tiếp theo
                    if i < num_times - 1:
                        # Kiểm tra và click "Gửi phản hồi khác"
                        self.check_for_submit_another_response()
                        
                        # Delay trước khi điền lần tiếp theo
                        if delay_between_forms > 0:
                            print(f"Chờ {delay_between_forms} giây...")
                            time.sleep(delay_between_forms)
                else:
                    print(f"✗ Lần {i + 1} thất bại")
                    
                    # Thử refresh lại form
                    self.driver.get(self.form_url)
                    time.sleep(3)
                    
            except Exception as e:
                print(f"✗ Lỗi ở lần {i + 1}: {e}")
                # Thử refresh lại form
                try:
                    self.driver.get(self.form_url)
                    time.sleep(3)
                except:
                    pass
        
        print(f"\n=== KẾT QUẢ ===")
        print(f"Tổng số lần thành công: {success_count}/{num_times}")
        print(f"Tỷ lệ thành công: {success_count/num_times*100:.1f}%")
    
    def fill_form(self):
        """Điền toàn bộ form (giữ lại để tương thích ngược)"""
        print("Bắt đầu điền form...")
        
        # Mở form
        self.open_form()
        
        # Điền form một lần
        self.fill_form_once()
    
    def close(self):
        """Đóng browser"""
        self.driver.quit()

# Hàm tiện ích để điền form nhiều lần
def fill_google_form_multiple_times(form_url, num_times, delay_between_forms=3):
    """
    Hàm tiện ích để điền Google Form nhiều lần
    
    Args:
        form_url (str): URL của Google Form
        num_times (int): Số lần điền form
        delay_between_forms (int): Thời gian delay giữa các lần điền (giây)
    """
    filler = GoogleFormFiller(form_url)
    
    try:
        filler.fill_form_multiple_times(num_times, delay_between_forms)
        input("\nNhấn Enter để đóng browser...")
        
    except KeyboardInterrupt:
        print("\nDừng chương trình theo yêu cầu người dùng")
        
    except Exception as e:
        print(f"Lỗi: {e}")
        
    finally:
        filler.close()

# Sử dụng
if __name__ == "__main__":
    # Thay URL form của bạn vào đây
    FORM_URL = "https://docs.google.com/forms/d/e/1FAIpQLScL2puWWWKJ5s8D3eeD3KZ7pbDsAY8X4N-40WrZso1a2_SwbQ/viewform"
    
    # Điền 200 lần với delay 2 giây
    fill_google_form_multiple_times(FORM_URL, 200, 2)

Bắt đầu điền form 200 lần...
Tỷ lệ được áp dụng:
- Giới tính: Nữ 84%, Khác 16%
- Độ tuổi: 20-25 (35%), 26-35 (40%), Khác (25%)
- Con nhỏ dưới 3 tuổi: Có 92%, Không 8%
- Kênh mua sản phẩm: Shopee 63%, Facebook 17%, Cửa hàng 10%, Website 10%
- Đã nghe Kamidi: Chưa nghe 13%, Đã nghe 62%, Đã sử dụng 25%
- Quảng cáo Kamidi: Gây khó chịu 5%, Khác 95%
--------------------------------------------------

--- Lần 1/200 ---
Chọn 'Nữ' (84% probability)
Đã chọn: Nữ
Chọn 'Dưới 20' (25% probability)
Đã chọn: Dưới 20
Chọn 'Có' cho câu hỏi con nhỏ (92% probability)
Đã chọn: Có
Chọn 'Chưa từng nghe' cho Kamidi (13% probability)
Đã chọn: Chưa từng nghe
Chọn 'Bình thường' cho quảng cáo Kamidi (34% probability)
Đã chọn: Bình thường
Đã chọn: Có
Đã chọn: Có
Đã chọn: Rất đồng ý
Đã chọn: Rồi
Đã chọn: Không hài lòng
Đã chọn: Không
Đã chọn: Không có ý kiến
Đã chọn: Tiktok
Chọn 'Không' cho xem livestream/video Kamidi (13% probability)
Đã chọn: Không
Đã chọn: 3- Biết sơ lược Kamidi là thương hiệu mẹ & bé
Chọn 'Sho